In [10]:
#importing the required libraries and authentication
import tweepy,json
access_token="2427460241-yihCbhCrkA6QrS7mhtwkK9FCnoKMvZzPNRFEtYr"
access_token_secret="p6dTQRMwy0SxVP49oHLUXeJ1L4T2cDqPM4RNlcEPErJ9X"
consumer_key="3nHkUhoqNif1x64w2gN7UxfPD"
consumer_secret="SD0qR7yWtUdGDqYxPcUpyNsRcMsX5MJB9z4MxjVKyq6VYwvDp0"
auth= tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth)

In [11]:
#search keywords for extracting tweets
search_terms = ['home-cooked food for corona patients','homemade food for corona patients','food for corona patients','food for covid patients', 'covid food delivery', 'covid free home cooked food delivery','homemade food covid','homemade food corona','home-cooked food covid','home-cooked food corona']
data = [] 
def stream_tweets(search_term):
    counter = 0 
    for tweet in tweepy.Cursor(api.search, q='\"{}\" -filter:retweets'.format(search_term), count=100, lang='en', tweet_mode='extended').items():
        tweet_details = {}
        tweet_details['time'] = str(tweet.user.created_at)
        tweet_details['id'] = tweet.user.id
        tweet_details['name'] = tweet.user.screen_name
        tweet_details['tweet'] = tweet.full_text
        tweet_details['retweets'] = tweet.retweet_count
        tweet_details['location'] = tweet.user.location
        tweet_details['created'] = tweet.created_at.strftime("%d-%b-%Y")
        tweet_details['followers'] = tweet.user.followers_count
        tweet_details['is_user_verified'] = tweet.user.verified
        data.append(tweet_details)
        counter += 1
        if counter == 1000:
            break
        else:
            pass
    with open('data.json', 'w') as f:
        json.dump(data, f)
    print('done!')

In [12]:
if __name__ == "__main__":
    print('Starting to stream...')
    for k in search_terms:
        stream_tweets(k)
    print('finished!')

Starting to stream...
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
finished!


In [13]:
#converting to csv for simplicity
import pandas as pd
df = pd.read_json('/content/data.json')

df.to_csv('/content/data_tweet.csv',index=None)

In [14]:
import pandas as pd
Data = pd.read_csv('/content/data_tweet.csv',index_col=False)


Data1 = Data.where(pd.notnull(Data), None)

Data1.head()


,time,id,name,tweet,retweets,location,created,followers,is_user_verified
0,2018-09-13 05:41:19,1040113174593163264,Iamsainani,Free food for corona patients in vijayawada \n...,1,Khammam,30-Apr-2021,581,False
1,2012-09-14 14:31:30,823403515,Robojohn11,Ground Report - TamiNadu Govt Hosp's - Food wh...,1,"Chennai, India",28-Apr-2021,234,False
2,2012-05-20 11:54:52,585611074,krishna23tiwari,"The BJP/BJYM begins a doctor's helpline, medic...",0,New Delhi & Mumbai,28-Apr-2021,8,False
3,2020-04-23 10:10:36,1253264950606221312,dhfm_4005,Food for corona patients for free in Vijayawad...,4,None,28-Apr-2021,241,False
4,2020-08-15 07:51:49,1294542222344376322,aashi0309,#COVIDEmergency2021 \nHome made food for Coron...,8,India,27-Apr-2021,552,False


In [ ]:
#creating connection to mysql and creating db
import mysql.connector as msql
from mysql.connector import Error
try:
    conn = msql.connect(host='sql129.main-hosting.eu', user='u291509283_cargill',  
                        password='Cargill123')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("CREATE DATABASE u291509283_cargill")
        print("irisDB database is created")
except Error as e:
    print("Error while connecting to MySQL", e)

In [ ]:
#updation in database
try:
    conn = msql.connect(host='sql129.main-hosting.eu', 
                           database='u291509283_cargill', user='u291509283_cargill', 
                           password='Cargill123')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        cursor.execute('DROP TABLE IF EXISTS Tweet_data;')
        print('Creating table....')
        cursor.execute("CREATE TABLE Tweet_data (time DATETIME,user_id BIGINT,name VARCHAR(255), tweet VARCHAR(255) ,retweets INTEGER,location VARCHAR(255),created VARCHAR(255),followers INTEGER,is_user_verified VARCHAR(255))")
        print("Tweet table is created....")
        for i,row in Data1.iterrows():
            sql = "INSERT INTO u291509283_cargill.Tweet_data VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row))
            print("Record inserted")
            
            conn.commit()
        
except Error as e:
    print("Error while connecting to MySQL", e)